In [1]:
import tools

import Bio
from Bio.PDB import PDBParser
from Bio.PDB.Chain import Chain

import py3Dmol

import warnings

from typing import List

In [2]:
# Load example, seeing interactions between A & B chain
warnings.filterwarnings("ignore", category=Bio.PDB.PDBExceptions.PDBConstructionWarning)

parser = PDBParser()
structure = parser.get_structure("9fg1", "9fg1.pdb")

structure

<Structure id=9fg1>

In [3]:
# Get A & B chains
A: Chain = structure[0]['A']
B: Chain = structure[0]['B']

In [ ]:
# View A & B chains
with open('9fg1.pdb', 'r') as protien:

  pdb_text = protien.read()

view = py3Dmol.view(width=800, height=800)

view.addModel(pdb_text)

view.setStyle({'cartoon': {'color': 'black'}})
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'red'}})
view.setStyle({'chain': 'B'}, {'cartoon': {'color': 'blue'}})

view.zoomTo()
view.show()

In [16]:
# Get interacting residues
# pseudonyms ribd, by_residue_distance, residue_interaction_by_distance
interacting_residues: List[tools.distance.InteractingResidue] = tools.by_residue_distance(A, B, 10.0)

In [17]:
# Return class structure for refrence
class Interaction:

  id: str
  distance: float
  
class InteractingResidue:

  chain: str
  position: int # in chain

  # to keep track of interactions
  id: str

  interactions: List[Interaction]

In [ ]:
# Show interacting residues

# view from before
for residue in interacting_residues:

    view.setStyle(
        {'chain': residue.chain, 'resi': residue.position},
        {'cartoon': {'color': 'yellow'}}
    )

view.zoomTo()
view.show()